In [1]:
# Perso
import sys
sys.path.append('/home/hugoperrin/Bureau/X/Cours/Data_camp/mars_craters/perso/')
from object_detector import *

import numpy as np
import pandas as pd

# Data
Xtrain = np.load('/home/hugoperrin/Bureau/X/Cours/Data_camp/mars_craters/data/data_train_mini.npy')
Ytrain = pd.read_csv('/home/hugoperrin/Bureau/X/Cours/Data_camp/mars_craters/data/labels_train_mini.csv').values

In [2]:
batches = CraterDataset(Xtrain, BATCH_SIZE, Ytrain)

In [3]:
for inputs, idx in batches.loader:
    break

In [4]:
inputs.shape

torch.Size([16, 3, 224, 224])

In [5]:
idx

tensor([ 7.,  5.,  2., 29., 17., 10., 11., 18., 25., 20., 13.,  9., 23., 21.,
         8., 14.])

In [6]:
net = create_SSD(config=config, pretrained=True, device='cuda:0')

In [7]:
LEARNING_RATE = 1e-6
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4

optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE,
                                        momentum=MOMENTUM,
                                        weight_decay=WEIGHT_DECAY)

In [9]:
net.train()

# Variable
inputs = Variable(inputs).to('cuda:0')

# Zero the parameter gradients
optimizer.zero_grad()

# Forward
loc, conf, prior = net(inputs)

loss = 0

In [10]:
# Loop on batch
image_idx = 0

In [11]:
# Get true craters
current_label_idx = idx[0]
true_circles = batches.Ytrain[batches.Ytrain[:, 0] == current_label_idx, 1:4] / 224 # Normalization to get same scale as priors
n_true = true_circles.size(0)
true_circles = Variable(true_circles).to('cuda:0')

In [29]:
true_circles

tensor([[0.6473, 0.3170, 0.0299],
        [0.3036, 0.3929, 0.0301]], device='cuda:0')

In [13]:
# Get prediction
predicted_loc = loc[image_idx]
predicted_conf = conf[image_idx]

In [20]:
# Matching
matches = match(prior, true_circles, threshold=0.4)
goal = encode_ground_truth(true_circles, prior, matches)

In [21]:
matches.max(dim=0)

(tensor([1., 1.], device='cuda:0'), tensor([485, 577], device='cuda:0'))

In [22]:
goal

tensor([[-1.6667,  1.6667, -9.8826],
        [ 0.0000,  2.2222, -9.8083],
        [ 0.0000, -2.2222, -9.8083],
        [-2.2222,  0.0000, -9.8083]], device='cuda:0')

In [ ]:
prior[456, :]

In [44]:
compute_IoU(prior, true_circles)[compute_IoU(prior, true_circles) > 0.4]

tensor([0.4438, 0.4635, 0.4635, 0.4635], device='cuda:0')

In [ ]:
predicted_loc[456]

In [26]:
(compute_IoU(prior, true_circles) > 0.35).squeeze()

tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]], device='cuda:0', dtype=torch.uint8)

In [36]:
decode_location(predicted_loc, prior)[(compute_IoU(prior, true_circles) > 0.35)[:, 1]] * 224

tensor([[67.6654, 83.8728,  9.8965],
        [67.6452, 83.5292, 17.1917],
        [67.8376, 91.5094,  9.2620],
        [69.3440, 91.5255, 16.7277],
        [71.9570, 87.9973,  9.9644],
        [71.9090, 87.9358, 18.0373]], device='cuda:0', grad_fn=<MulBackward>)

In [38]:
prior[(compute_IoU(prior, true_circles) > 0.35)[:, 1]] * 224

tensor([[68., 84.,  5.],
        [68., 84.,  9.],
        [68., 92.,  5.],
        [68., 92.,  9.],
        [72., 88.,  5.],
        [72., 88.,  9.]], device='cuda:0')

In [37]:
true_circles[1, :] * 224

tensor([68.0000, 88.0000,  6.7500], device='cuda:0')

In [39]:
target, indices = matches.max(dim=1)

In [41]:
positive_pred = predicted_conf[target == 1]
num_pos = positive_pred.size(0)

In [42]:
raw_negative_pred = predicted_conf[target == 0]
raw_num_neg = raw_negative_pred.size(0)

# Hard negative mining (reduce the ratio of negative over positive samples to 3:1)
num_neg = min(3 * num_pos, raw_num_neg)
negative_pred = raw_negative_pred.sort(dim=0, descending=True)[0][0:num_neg]

prediction = torch.cat([positive_pred, negative_pred]).view(-1).to('cuda:0')
target_conf = torch.cat([torch.ones(num_pos), torch.zeros(num_neg)]).view(-1).to('cuda:0')

In [43]:
prediction, target_conf

(tensor([ 0.6622, -0.1229, -0.1152,  0.0015,  1.5372,  1.5087,  1.4499,  1.3746,
          1.2654,  1.2596,  1.2561,  1.2348,  1.2309,  1.2293,  1.2257,  1.2210],
        device='cuda:0', grad_fn=<ViewBackward>),
 tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        device='cuda:0'))

In [ ]:
conf_loss = nn.BCEWithLogitsLoss(reduction='sum')
image_conf_loss = conf_loss(prediction, target_conf) / num_pos

In [ ]:
image_conf_loss

In [ ]:
nn.Sigmoid()(prediction)

In [ ]:
pred_loc = predicted_loc[target == 1]

In [ ]:
pred_loc

In [ ]:
loc_loss = F.smooth_l1_loss(pred_loc, goal, reduction='sum') / num_pos

In [ ]:
loc_loss

In [ ]:
# Image loss
image_loss = multi_circle_loss(predicted_loc, goal, predicted_conf, matches, 1)

image_loss

In [ ]:
image_conf_loss + loc_loss